<center><img src="images/DLI Header.png" alt="Header" width="400"></center>

# 7.0  Live Stream
### USB WEBCAM CONNECTED TO JETSON REQUIRED

In the examples presented so far, the input stream has been a file, played as a stream.  In this notebook, you'll use a live stream using a webcam. Attach a USB webcam to your Jetson Nano using an available USB port.  

<img src="images/06_example.png" alt="webcam input">

**[7.1 Build a Pipeline with Webcam Input](#7.1-Build-a-Pipeline-with-Webcam-Input)**<br>
&nbsp; &nbsp; &nbsp;[7.1.1 Practice Application `deepstream-test1-usbcam-rtsp-out`](#7.1.1-Practice-Application-deepstream-test1-usbcam-rtsp-out)<br>
&nbsp; &nbsp; &nbsp;[7.1.2 Exercise: Run the Base Application](#7.1.2-Exercise:-Run-the-Base-Application)<br>
**[7.2 Change the Network to YOLO](#7.2-Change-the-Network-to-YOLO)**<br>
&nbsp; &nbsp; &nbsp;[7.2.1 Exercise: Run YOLO with a Webcam](#7.2.1-Exercise:-Run-YOLO-with-a-Webcam)<br>

# 7.1 Build a Pipeline with Webcam Input
In all of our applications so far, we have used files as our input sources.  The pipelines processed these files in two ways before passing the streams to the `Gst-nvstreammux` plugin:

1. Single file with known format as in the `deepstream-test1` and `deepstream-test2` sample apps:
  - `GstFileSrc` - reads the video data from file
  - `GstH264Parse` - parses the incoming H264 stream
  - `Gst-nvv4l2decoder` - hardware accelerated decoder; decodes video streams using NVDEC
  - `Gst-nvstreammux` - batch video streams before sending for AI inference   
  
2. Multistream input using URI with run-time determination of format as in the `deepstream-test3` sample app; note that we could have used URI streams other than "file" in this case:
  - `uridecoderbin` source bins - one or more instances created at runtime to read the video data
  - `Gst-nvstreammux` - batch video streams before sending for AI inference   

For the webcam input, the `deepstream-test1-usbcam` sample app uses a different set of plugins at the beginning of the pipeline. 

3. Video capture device input with conversion:
  - `GstV4l2Src` - can be used to capture video from v4l2 devices, like webcams and tv cards
  - `GstCapsFilter` - enforces limitations on data (no data modification)
  - `GstVideoConvert` - Convert video frames between a great variety of video formats
  - `Gst-nvvideoconvert` - performs video color format conversion (I420 to RGBA)
  - `GstCapsFilter` - enforces limitations on data (no data modification)
  - `Gst-nvstreammux` - batch video streams before sending for AI inference

We'll begin with the `deepstream-test1-usbcam-rtsp-out` [course application](deepstream/sources/deepstream_python_apps/dli_apps/deepstream-test1-usbcam-rtsp-out/deepstream_test_1_usb.py), which is a combination of the `deepstream-test1-usbcam` and `deepstream-test1-rstp-out` DeepStream Python sample applications (found at `deepstream/sources/deepstream_python_apps/apps`). 

Here's a look at the pipeline elements created in the application up to `nvstreammux`, which is common to all of our apps:
```python
    print("Creating Source \n ")
    source = Gst.ElementFactory.make("v4l2src", "usb-cam-source")
    if not source:
        sys.stderr.write(" Unable to create Source \n")

    caps_v4l2src = Gst.ElementFactory.make("capsfilter", "v4l2src_caps")
    if not caps_v4l2src:
        sys.stderr.write(" Unable to create v4l2src capsfilter \n")

    # videoconvert to make sure a superset of raw formats are supported
    vidconvsrc = Gst.ElementFactory.make("videoconvert", "convertor_src1")
    if not vidconvsrc:
        sys.stderr.write(" Unable to create videoconvert \n")

    # nvvideoconvert to convert incoming raw buffers to NVMM Mem (NvBufSurface API)
    nvvidconvsrc = Gst.ElementFactory.make("nvvideoconvert", "convertor_src2")
    if not nvvidconvsrc:
        sys.stderr.write(" Unable to create Nvvideoconvert \n")

    caps_vidconvsrc = Gst.ElementFactory.make("capsfilter", "nvmm_caps")
    if not caps_vidconvsrc:
        sys.stderr.write(" Unable to create capsfilter \n")

    # Create nvstreammux instance to form batches from one or more sources.
    streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
    if not streammux:
        sys.stderr.write(" Unable to create NvStreamMux \n")

```    

In summary, the pipeline for this app consists of the following plugins (ordered):

- `GstV4l2Src` - can be used to capture video from v4l2 devices, like webcams and tv cards
- `GstCapsFilter` - enforces limitations on data (no data modification)
- `GstVideoConvert` - Convert video frames between a great variety of video formats
- `Gst-nvvideoconvert` - performs video color format conversion (I420 to RGBA)
- `GstCapsFilter` - enforces limitations on data (no data modification)
- `Gst-nvstreammux` - batch video streams before sending for AI inference
- `Gst-nvinfer` - runs inference using TensorRT
- `Gst-nvvideoconvert` - performs video color format conversion (I420 to RGBA)
- `Gst-nvdsosd` - draw bounding boxes, text and region of interest (ROI) polygons
- `Gst-nvvideoconvert` - performs video color format conversion (RGBA to I420)
- `GstCapsFilter` - enforces limitations on data (no data modification)
- `Gst-nvv4l2h264enc` - encodes RAW data in I420 format to H264
- `GstRtpH264Pay` - converts H264 encoded Payload to RTP packets (RFC 3984)
- `GstUDPSink` - sends UDP packets to the network. When paired with RTP payloader (`Gst-rtph264pay`) it can implement RTP streaming

# 7.1.1 Practice Application `deepstream-test1-usbcam-rtsp-out`

A sample app, based on the `deepstream-test1` reference app, receives input from a USB webcam and outputs to an RTSP stream.

In [ ]:
# Set some path locations for readability
PYTHON_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/apps'
STREAMS = '/opt/nvidia/deepstream/deepstream/samples/streams'
DLI_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/dli_apps'

# List the contents of the sample app
!ls $DLI_APPS/deepstream-test1-usbcam-rtsp-out

## 7.1.2 Exercise: Run the Base Application
Plug in your webcam and execute the following cells. 

In [ ]:
# Check usage of the test3 mp4 app with the help option
!cd $DLI_APPS/deepstream-test1-usbcam-rtsp-out \
    && python3 deepstream_test_1_usb.py --help

#### Run the DeepStream app
Open the VLC media player on your laptop:
- Pull down the "Media" menu and select the "Open Network Stream" dialog.
- Set the URL to `rtsp://192.168.55.1:8554/ds-test`.
- Optionally, add a wait delay to VLC:
   - Click "Show more options" in the dialog.
   - Add ` :ipv4=120000` to the "Edit Options" line to add a 120 second delay.
- Start execution of the cell below.
- Click "Play" on your VLC media player *after* you start the cell execution.  

The stream will start from the Jetson Nano and display in the media player.  There is a delay while the model `.engine` file is built.  

If VLC fails, start it again. Close the VLC fail notice and press the "play" triangle.

To end the stream, use the pull-down menu to interrupt the kernel with *Kernel->Interrupt Kernel*.

In [ ]:
# Run the app
!cd $DLI_APPS/deepstream-test1-usbcam-rtsp-out \
    && python3 deepstream_test_1_usb.py -i /dev/video0

To see if the app is working properly, bring an object into the webcam frame that is either a vehicle, person, bicycle, or road sign, as that is what the PGIE model we're using recognizes.  In the image shown here, the webcam was pointed at a car magazine website for testing.

<img src="images/06_usb.png">

# 7.2 Change the Network to YOLO
If you set up the YOLO network in the optional [Using Different Networks](05_DiffNetworks.ipynb) notebook, you can change models and use YOLO to identify many more objects with your webcam (up to 80!).  The `deepstream-test1-usbcam-yolo-rtsp-out` application is very similar to the `deepstream-test1-usbcam-rtsp-out` app, but configured for YOLO. It uses the YOLO config file and references that from the Python file.  In addition, variables and constants related to the previous model have been commented away. To see the actual difference in lines, execute the following `diff` command.

In [ ]:
!cd $DLI_APPS \
    && diff deepstream-test1-usbcam-rtsp-out/deepstream_test_1_usb.py \
            deepstream-test1-usbcam-yolo-rtsp-out/deepstream_test_1_usb.py

# 7.2.1 Exercise: Run YOLO with a Webcam
1. Ensure that you have YOLO installed.  
   - If you need to reinstall YOLO, connect your Jetson to the Internet and execute the cells in the [Using Different Networks](05_DiffNetworks.ipynb) notebook up to the first exercise.
1. Execute the application as before, using your RTSP viewer to see the webcam stream.  
   - There will be a delay while the `.engine` file is built and you may need to restart the RTSP viewer.  
1. Try pointing your webcam at various objects to see if the object detector recognizes them.

In [ ]:
# Run the app
!cd $DLI_APPS/deepstream-test1-usbcam-yolo-rtsp-out \
    && python3 deepstream_test_1_usb.py -i /dev/video0

#### How did you do?
The YOLO network has more objects it can detect, but you may have to experiment a bit with the threshold levels in the configuration file for optimal results for your environment and objects.

<img src="images/06_usb_yolo.png" alt="file output YOLO">

<h2 style="color:green;">Congratulations!</h2>

You've completed all the notebooks.  Be sure to work through the assessment questions in the online portion of the DLI course to get your certificate!

<center><img src="images/DLI Header.png" alt="Header" width="400"></center>